In [15]:
import pandas as pd

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [18]:
train['SMILES'].value_counts()

COCCOc1cc(=O)n2c(c1C(=O)N1CCCC1)CCOCC2                                                                                                                                            2
Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4cc(Cl)c(Cl)cc43)cc2)cn1                                                                                                             2
O=S(=O)(c1ccccc1)n1ccc(C/C=C/Cc2ccn(S(=O)(=O)c3ccccc3)c2)c1                                                                                                                       2
COc1cc(CN2CCC(NC(=O)c3cc4cc(Br)c5ccccc5c4o3)CC2)cc(OC)c1OC                                                                                                                        2
COc1cccc(-c2cc(-c3nc4c5cnn(-c6ccccc6)c5ncn4n3)n[nH]2)c1                                                                                                                           2
                                                                                                    

In [6]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train = train.drop(['id','HLM'],axis=1)
test = test.drop('id',axis=1)

In [7]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label='MLM').fit(train.loc[:3000,:])

No path specified. Models will be saved in: "AutogluonModels\ag-20230902_103856"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230902_103856"
AutoGluon Version:  0.8.2
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   283.02 GB / 494.27 GB (57.3%)
Train Data Rows:    3001
Train Data Columns: 8
Label Column: MLM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (131.72, 0.0, 37.70528, 35.84407)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                 

In [8]:
predictor.evaluate(train.loc[3000:,:], silent=True)

{'root_mean_squared_error': -31.25162053253611,
 'mean_squared_error': -976.6637859096326,
 'mean_absolute_error': -25.819749847409234,
 'r2': 0.18890309292688579,
 'pearsonr': 0.4427417239537802,
 'median_absolute_error': -23.34752600097656}

In [9]:
predictor.leaderboard(train.loc[3000:,:], silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-31.251621,-31.520862,0.277693,0.193458,101.881199,0.007137,0.001010,0.785652,2,True,12
1,NeuralNetFastAI,-31.341976,-31.596485,0.114007,0.079190,22.578340,0.114007,0.079190,22.578340,1,True,8
2,CatBoost,-31.620230,-32.041374,0.036443,0.006068,18.824924,0.036443,0.006068,18.824924,1,True,6
3,XGBoost,-31.812976,-32.488286,0.067494,0.023566,1.217651,0.067494,0.023566,1.217651,1,True,9
4,LightGBMLarge,-31.879452,-32.754135,0.042364,0.010963,2.235707,0.042364,0.010963,2.235707,1,True,11
5,LightGBM,-31.913968,-32.379669,0.017139,0.002990,0.275680,0.017139,0.002990,0.275680,1,True,4
6,LightGBMXT,-31.986993,-32.140848,0.044804,0.004983,0.983728,0.044804,0.004983,0.983728,1,True,3
7,RandomForestMSE,-32.143914,-33.044602,0.357969,0.034459,0.964044,0.357969,0.034459,0.964044,1,True,5
8,ExtraTreesMSE,-32.280507,-32.829304,0.356443,0.132038,1.598659,0.356443,0.132038,1.598659,1,True,7
9,NeuralNetTorch,-33.591511,-33.424841,0.102968,0.104200,59.416603,0.102968,0.104200,59.416603,1,True,10


In [10]:
mlm = predictor.predict(test)

In [11]:
from autogluon.tabular import TabularDataset, TabularPredictor

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train = train.drop(['id','MLM'],axis=1)
test = test.drop('id',axis=1)

predictor = TabularPredictor(label='HLM').fit(train.loc[:3000,:])
print(predictor.evaluate(train.loc[3000:,:], silent=True))
predictor.leaderboard(train.loc[3000:,:], silent=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20230902_104049"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230902_104049"
AutoGluon Version:  0.8.2
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   282.90 GB / 494.27 GB (57.2%)
Train Data Rows:    3001
Train Data Columns: 8
Label Column: HLM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (135.336, 0.0, 53.21124, 36.04317)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                

{'root_mean_squared_error': -32.28947360273081, 'mean_squared_error': -1042.61010554145, 'mean_absolute_error': -27.38325745885152, 'r2': 0.20829364263818662, 'pearsonr': 0.4571204129940179, 'median_absolute_error': -25.72664723205567}


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-32.289474,-31.341187,0.524974,0.238696,49.209935,0.007972,0.000998,0.782051,2,True,12
1,XGBoost,-32.344926,-32.270293,0.064663,0.020139,1.207591,0.064663,0.020139,1.207591,1,True,9
2,LightGBMXT,-32.386738,-31.851416,0.064295,0.024611,1.185797,0.064295,0.024611,1.185797,1,True,3
3,LightGBM,-32.646679,-32.062884,0.023583,0.011494,0.844401,0.023583,0.011494,0.844401,1,True,4
4,NeuralNetFastAI,-32.671391,-31.914637,0.107579,0.077530,20.547576,0.107579,0.077530,20.547576,1,True,8
5,RandomForestMSE,-32.676248,-32.027613,0.354827,0.132841,3.432089,0.354827,0.132841,3.432089,1,True,5
6,ExtraTreesMSE,-32.715665,-32.247398,0.349262,0.135926,1.598622,0.349262,0.135926,1.598622,1,True,7
7,CatBoost,-32.720616,-31.550724,0.031906,0.008997,24.425573,0.031906,0.008997,24.425573,1,True,6
8,LightGBMLarge,-32.762294,-32.228026,0.031820,0.012263,2.322546,0.031820,0.012263,2.322546,1,True,11
9,NeuralNetTorch,-33.079327,-33.301682,0.113011,0.100332,63.835247,0.113011,0.100332,63.835247,1,True,10


In [12]:
hlm = predictor.predict(test)

In [14]:
sub = pd.read_csv('./sample_submission.csv')
sub['MLM'] = mlm
sub['HLM'] = hlm
sub = sub.set_index('id')
sub.to_csv('./submission.csv')